In [2]:
# How to call the correct_lean function
from postprocess_original_utils import correct_lean

correct_lean(_human_dataset)

In [19]:
# How to reprocess the data instead of loading from a save
from postprocess_original_utils import load_data

data = load_data(load_from_file=False)

Read in 601 HIT total assignments
568 good, 33 flagged, 0 error, 0 rejected
Relative depth ordering for Turkers associated with the data
105812 images in human3.6 dataset
568 annotations matched with ground truth
Output data to /Users/Robert/Documents/Caltech/CS81_Depth_Research/results/human36m_processed_data.json


In [50]:
# Load results from caltech_results into python
import json

OLD_KEYCMPS = "/Users/Robert/Documents/Caltech/EE148/ee148project/keycmps.json"
CALTECH_RESULTS = "/Users/Robert/Documents/Caltech/CS81_Depth_Research/results/caltech_results.json"

caltech_results = json.load(open(CALTECH_RESULTS))
old_keycmps = json.load(open(OLD_KEYCMPS))

In [51]:
# Convert caltech results into something consumable by the local server to get the mergesort ordering
import json
KEYCMPS_RESULT_PATH = "/Users/Robert/Documents/Caltech/CS81_Depth_Research/results/caltech_results_keycmps.json"

temp = [map(json.loads, c['_hit_depth_str'].split("***")[1:]) for c in caltech_results]
keycmps = [item for t in temp for item in t]

keycmp_orders = [k['keypoint_comparisons_order'] for k in keycmps]
keycmp_reses = [k['keypoint_comparisons_res'] for k in keycmps]
keycmps = {'keycmp_orders': keycmp_orders, 'keycmp_reses': keycmp_reses}

with open(KEYCMPS_RESULT_PATH, 'w') as f:
    f.write(json.dumps(keycmps))

In [52]:
import json
# All the comparisons and hits for one person are squished into one hit. Separate them into multiple hits.
unfolded_caltech_results = []
temp = [map(json.loads, c['_hit_depth_str'].split("***")[1:]) for c in caltech_results]
for t, c in zip(temp, caltech_results):
    for trial_ind, cmps in enumerate(t):
        res_copy = c.copy()
        keycmp_order = cmps['keypoint_comparisons_order']
        keycmp_res = cmps['keypoint_comparisons_res']
        res_copy['keypoint_comparisons_order'] = keycmp_order
        res_copy['keypoint_comparisons_res'] = keycmp_res
        res_copy['trial_ind'] = trial_ind
        unfolded_caltech_results.append(res_copy)

# Take the mergesort ordering from the local server and reincorporate it into the original data
KEYPTS_RELATIVE_DEPTH_PATH = "/Users/Robert/Documents/Caltech/CS81_Depth_Research/results/caltech_results_keypts_relative_depth.json"

_relative_depth = json.load(open(KEYPTS_RELATIVE_DEPTH_PATH, 'r'))
for d, kpts_relative_depth in zip(unfolded_caltech_results, _relative_depth):
    d['kpts_relative_depth'] = kpts_relative_depth

In [54]:
# For the newly separated hits, the img_id must be reassociated with it
# All trial_inds above 0 should line up with this list
miniexp_img_ids = [56833, 965922, 849671, 649263, 12750, 896082, 571176, 965922]

for u in unfolded_caltech_results:
    if u['trial_ind'] > 0:
        u['img_id'] = miniexp_img_ids[u['trial_ind']]

In [61]:
# Time to line up the hits with the ground truth human data. This is more or less the same as the 
# code from the original load data postprocess.
import json
import copy
from postprocess_original_utils import correct_lean
from constants import HUMAN_ANNOTATION_PATH

img_ids = [c['img_id'] for c in unfolded_caltech_results]

# Get the ground truth annotations from human dataset
with open(HUMAN_ANNOTATION_PATH) as f:
    _human_dataset = json.load(f)
    correct_lean(_human_dataset)

for i in range(len(_human_dataset['images'])):
    if _human_dataset['images'][i]['id'] in img_ids:
        img_id_idxes = [img_id_idx for img_id_idx, img_id in enumerate(img_ids)
                        if img_id == _human_dataset['images'][i]['id']]
        for img_id_idx in img_id_idxes:
            unfolded_caltech_results[img_id_idx]['images_truth'] = _human_dataset['images'][i]
            unfolded_caltech_results[img_id_idx]['annotations_truth'] = copy.deepcopy(_human_dataset['annotations'][i])

print "{} images in human3.6 dataset".format(len(_human_dataset['images']))
print "{} annotations matched with ground truth".format(len(unfolded_caltech_results))

# Remove the neck keypoint. Arrange the depth data in a useful way.
for d in unfolded_caltech_results:
    # Remove neck keypoint
    d['annotations_truth']['kpts_2d'][2:4] = []
    d['annotations_truth']['kpts_3d'][3:6] = []
    # Grab every third kpts_3d which corresponds to depth
    axis_to_sort = 1 # change this if it looks like we're sorting along the wrong axis
    d['annotations_truth']['kpts_depth'] = d['annotations_truth']['kpts_3d'][axis_to_sort::3]
    d['annotations_truth']['kpts_relative_depth'] = \
        [i[0] for i in sorted(enumerate(d['annotations_truth']['kpts_depth']), key=lambda x:x[1])]


105812 images in human3.6 dataset
78 annotations matched with ground truth


In [63]:
# Finally write it to a file!
import json
CALTECH_OUTPUT_PATH = "/Users/Robert/Documents/Caltech/CS81_Depth_Research/results/human36m_processed_caltech_data.json"

with open(CALTECH_OUTPUT_PATH, 'w') as f:
    json.dump(unfolded_caltech_results, f)
print "Output data to {}".format(CALTECH_OUTPUT_PATH)

Output data to /Users/Robert/Documents/Caltech/CS81_Depth_Research/results/human36m_processed_caltech_data.json


In [7]:
import json
from constants import CALTECH_OUTPUT_PATH
data = json.load(open(CALTECH_OUTPUT_PATH, 'r'))

data[3]['annotations_truth']['i_id']
set([d['_hit_comment'][:20] + ": " + d['_worker_id'] for d in data])
[d for d in data if d['_worker_id'] == 'nonAMT_368102']

[{u'_assignment_id': u'',
  u'_gui_rating': 4,
  u'_hit_comment': u'Matteo',
  u'_hit_depth_str': u'***{"keypoint_comparisons_unsure":[],"keypoint_comparisons_res":{"1,2":-1,"2,1":1,"1,0":-1,"0,1":1,"2,0":1,"0,2":-1,"4,5":1,"5,4":-1,"3,5":0,"5,3":0,"3,4":-1,"4,3":1,"1,5":0,"5,1":0,"1,4":-1,"4,1":1,"0,4":-1,"4,0":1,"2,4":-1,"4,2":1,"7,8":-1,"8,7":1,"7,6":-1,"6,7":1,"1,3":0,"3,1":0,"6,8":1,"8,6":-1,"9,10":-1,"10,9":1,"11,12":-1,"12,11":1,"11,9":0,"9,11":0,"12,9":1,"9,12":-1,"12,10":0,"10,12":0,"7,11":0,"11,7":0,"7,12":-1,"12,7":1,"8,12":0,"12,8":0,"5,11":0,"11,5":0,"7,9":0,"9,7":0,"8,10":0,"10,8":0,"5,9":0,"9,5":0,"3,11":0,"11,3":0,"3,9":0,"9,3":0,"3,7":0,"7,3":0,"1,11":0,"11,1":0,"1,9":0,"9,1":0,"1,7":0,"7,1":0,"7,5":0,"5,7":0,"5,12":-1,"12,5":1,"3,12":-1,"12,3":1,"1,12":-1,"12,1":1,"5,10":-1,"10,5":1,"5,8":-1,"8,5":1,"0,12":-1,"12,0":1,"0,10":-1,"10,0":1,"0,8":-1,"8,0":1,"2,12":0,"12,2":0,"10,2":0,"2,10":0,"8,2":0,"2,8":0,"2,6":-1,"6,2":1,"12,6":-1,"6,12":1,"10,6":-1,"6,10":1,"4,6":-1,